In [ ]:
from __future__ import print_function, division
import numpy as np
import json
import time
import scipy
import scipy.sparse
from trimesh import TriMesh
import glob
from numpy import *
np.set_printoptions(linewidth=2000, suppress=True)
from Extract_Transformation_matrix_minimize_SVD_singular_value_allPoses import *

In [ ]:
#### read obj file
base_dir="./datas/cube4/poses-1/"
# base_dir="../models/cube4/poses-1/"
filenames=glob.glob(base_dir+"*.obj")
print(filenames)
meshes_List=[]
for i in range(len(filenames)):
    meshes_List.append(TriMesh.FromOBJ_FileName(filenames[i]))

mesh0=meshes_List[0]
mesh1_list=meshes_List[1:]

groundtruth_names=glob.glob(base_dir+"cube-*.DMAT")
print(groundtruth_names)

gt_all=None
for i in range(len(groundtruth_names)):
    gt=load_DMAT(groundtruth_names[i])
    gt=gt.T
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_all is None:
        gt_all=gt
    else:
        gt_all=np.hstack((gt_all, gt))
    

print(gt_all.shape)

ind=np.arange(10)


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 

P = len(mesh1_list)
M=12*P

In [ ]:
#### solve for transformation per vertex, with smoothness regularization and transformation matrix L2 norm regularization.
output_prefix=base_dir+"Vertex_tramsformation_matrix_recovering"
weights = {'W_svd': 2.0, 
           'W_rotation': 0.01, 
           'W_rotation1': 0.0, 
           'W_rotation2': 0.0, 
           'W_translation':0.0, 
           'W_spatial': 0.0
          }


start=time.time()

print("solving transformation matrix")



x0=np.ones(len(vertices1)*M)/M
    
# x0=np.zeros(len(vertices1)*M)

# x0=gt_all.ravel()

# #### use identity matrix as input
# xi=np.hstack((np.identity(3),np.zeros((3,1)))).reshape((1,-1))
# print xi
# x0=np.repeat(xi, len(vertices0), 0).reshape(-1)


res=run_one(mesh0, mesh1_list, output_prefix, weights, initials=x0, option=3)
# res=run_one(mesh0, mesh1_list, output_prefix, weights, initials=x0, option=4)### binary choose nonzero svd singlular value number
res=res.reshape((len(mesh0.vs),-1))
end=time.time()
print("using time: ", end-start)
print(res.shape)

In [ ]:
print res.shape
L=res-res.mean(axis=0).reshape((1,-1))

eigenVal,eigenVec=np.linalg.eig(L.T.dot(L))

print(sorted(eigenVal))

In [ ]:
gtL=gt_all-gt_all.mean(axis=0).reshape((1,-1))

gteigenVal,gteigenVec=np.linalg.eig(gtL.T.dot(gtL))

print(sorted(gteigenVal))

In [ ]:
print(res.shape)

L=res-res.mean(axis=0).reshape((1,-1))

s = np.linalg.svd(L, full_matrices=True, compute_uv=False)
print(s)

In [ ]:
gtL=gt_all-gt_all.mean(axis=0).reshape((1,-1))

s = np.linalg.svd(gtL, full_matrices=True, compute_uv=False)
print(s)

In [ ]:
temp1=res.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
identities=np.repeat(np.identity(3).reshape((1,-1)), len(temp1), 0).ravel()
print(abs((temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel()-identities).max())

In [ ]:
temp1=gt_all.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
identities=np.repeat(np.identity(3).reshape((1,-1)), len(temp1), 0).ravel()
print(abs((temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel()-identities).max())

In [ ]:
#### check non_diagnal_sum of RTR is 0.
temp1=res.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
RTR=(temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel() #### per vertex R^T*R.

RTRsqure=np.square(RTR)
inds=np.repeat(np.array([[0,4,8]]), len(temp1), 0)+ np.arange(len(temp1)).reshape((-1,1))*9  #### diganl element index in ravel array.

diagnal_term=np.square(RTR[inds.ravel()]-np.ones(len(inds.ravel()))).sum()
non_diagnal_sum_square=RTRsqure.sum()-RTRsqure[inds.ravel()].sum()

print(diagnal_term)
print(non_diagnal_sum_square)

all_inds=np.arange(len(temp1)*9)
non_diagnal_inds=np.asarray(list(set(list(all_inds))-set(list(inds.ravel()))))

print(abs(RTR[non_diagnal_inds]).max())
print(np.median(abs(RTR[non_diagnal_inds])))
print(RTR[inds.ravel()].max())
print(RTR[inds.ravel()].min())

In [ ]:
#### check non_diagnal_sum of RTR is 0.
temp1=gt_all.reshape((-1,3,4))[:,:,:3]
temp2=temp1.transpose((0,2,1))
RTR=(temp1[:,:,:,np.newaxis]*temp2[:,np.newaxis,:,:]).sum(axis=-2).ravel() #### per vertex R^T*R.

RTRsqure=np.square(RTR)
inds=np.repeat(np.array([[0,4,8]]), len(temp1), 0)+ np.arange(len(temp1)).reshape((-1,1))*9  #### diganl element index in ravel array.

diagnal_term=np.square(RTR[inds.ravel()]-np.ones(len(inds.ravel()))).sum()
non_diagnal_sum_square=RTRsqure.sum()-RTRsqure[inds.ravel()].sum()

print(diagnal_term)
print(non_diagnal_sum_square)

all_inds=np.arange(len(temp1)*9)
non_diagnal_inds=np.asarray(list(set(list(all_inds))-set(list(inds.ravel()))))

print(abs(RTR[non_diagnal_inds]).max())
print(np.median(abs(RTR[non_diagnal_inds])))
print(RTR[inds.ravel()].max())
print(RTR[inds.ravel()].min())

In [ ]:
print(abs(res-gt_all).max())
print(res.ravel()[abs(res-gt_all).argmax()])
print(gt_all.ravel()[abs(res-gt_all).argmax()])
print(np.sqrt(np.square(res-gt_all).mean()))

In [ ]:
########## visualize difference of recovered M and groundtruth M.
import matplotlib.pyplot as plt
import matplotlib.animation
from mpl_toolkits.mplot3d import Axes3D

fig= plt.figure()
ax=fig.add_subplot(111, projection='3d')
text = fig.suptitle('transformation matrix comparison')

test_vals=np.array([
        [0,0,0,1],
        [1,0,0,1],
        [0,1,0,1],
        [0,0,1,1]
    ])

lb=-2
ub=2

recover_vals_list=[]
gt_vals_list=[]

res=res.reshape((-1, 12))
gt_all=gt_all.reshape((-1, 12))


frames=len(res)
for num in range(frames):
    resi=res[num]
    gti=gt_all[num]
    recover_vals=np.multiply(resi.reshape((1,3,4)), test_vals.reshape((-1,1,4))).sum(axis=-1)
    gt_vals=np.multiply(gti.reshape((1,3,4)), test_vals.reshape((-1,1,4))).sum(axis=-1)
    recover_vals_list.append(recover_vals)
    gt_vals_list.append(gt_vals)


def update_graph(num):
    ax.clear()##### if you want to show accumulated data in one figure, comment this line.
    
    recover_vals=recover_vals_list[num]
    gt_vals=gt_vals_list[num]
    
    
    
    for u1, v1, w1 in zip(test_vals[1:,0]-test_vals[0,0], test_vals[1:,1]-test_vals[0,1], test_vals[1:,2]-test_vals[0,2]):
        graph=ax.quiver(test_vals[0,0], test_vals[0,1], test_vals[0,2], u1, v1, w1, pivot = 'tail', length=np.sqrt(u1**2+v1**2+w1**2), color='r')
    
    
    for u1, v1, w1 in zip(recover_vals[1:,0]-recover_vals[0,0], recover_vals[1:,1]-recover_vals[0,1], recover_vals[1:,2]-recover_vals[0,2]):
        graph=ax.quiver(recover_vals[0,0], recover_vals[0,1], recover_vals[0,2], u1, v1, w1, pivot = 'tail', length=np.sqrt(u1**2+v1**2+w1**2), color='g')
    
    for u1, v1, w1 in zip(gt_vals[1:,0]-gt_vals[0,0], gt_vals[1:,1]-gt_vals[0,1], gt_vals[1:,2]-gt_vals[0,2]):
        graph=ax.quiver(gt_vals[0,0], gt_vals[0,1], gt_vals[0,2], u1, v1, w1, pivot = 'tail', length=np.sqrt(u1**2+v1**2+w1**2), color='b')


    ax.set_xlim(lb,ub)
    ax.set_ylim(lb,ub)
    ax.set_zlim(lb,ub)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    text.set_text('transformation matrix comparison={}'.format(num))
    
    
    return text, graph


ani = matplotlib.animation.FuncAnimation(fig, update_graph, frames, interval=50)

plt.show()